In [33]:
using Convex
using Mosek

In [30]:
include("optimizer_proj.jl")

srand(1234)

# Generate some matrix
b = 10
num_blocks = 100
n = num_blocks*(b-1) + 1

A = spzeros(n, n)

for i = 1:num_blocks
    top_left = (i-1)*(b-1) + 1
    A[top_left:top_left+b-1, top_left:top_left+b-1] .= randn(b, b)
end

A = (A + A')/2;

all_A = SparseMatrixCSC{Float64, Int64}[]
push!(all_A, A)
for i = 1:10
    push!(all_A, -speye(size(A, 1)))
end
C = ones(1,10)
d = ones(1)

1-element Array{Float64,1}:
 1.0

In [31]:
@time optimize(all_A, C, d)

INFO: Initiating pre-solve
INFO: found feasible starting point with tol : 1.8254843040965696e-5
INFO: Starting solve
INFO: iteration 1. Duality gap = 0.010000000000000002, feasibility gap = 8.526512829121202e-14
INFO: iteration 2. Duality gap = 0.0010000000000000002, feasibility gap = 5.329070518200751e-15
INFO: iteration 3. Duality gap = 0.00010000000000000003, feasibility gap = 0.0


 10.865400 seconds (7.81 M allocations: 23.430 GiB, 16.88% gc time)


INFO: iteration 4. Duality gap = 1.0000000000000004e-5, feasibility gap = 1.1102230246251565e-16
INFO: iteration 5. Duality gap = 1.0000000000000004e-6, feasibility gap = 1.1102230246251565e-16
INFO: converged in 5 iterations.


11-element Array{Float64,1}:
 -7.89627
  0.1    
  0.1    
  0.1    
  0.1    
  0.1    
  0.1    
  0.1    
  0.1    
  0.1    
  0.1    

In [34]:
t = Variable()
x = Variable(length(all_A) - 1)

L = -t*speye(n)
for i = 2:length(all_A)
    L += all_A[i]*x[i-1]
end
L += all_A[1]
prob = minimize(-t, [isposdef(L), C*x == d])

Problem:
minimize AbstractExpr with
head: -
size: (1, 1)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()

subject to
Constraint:
sdp constraint
expression: AbstractExpr with
head: +
size: (901, 901)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()


		Constraint:
== constraint
lhs: AbstractExpr with
head: *
size: (1, 1)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()

rhs: [1.0]
vexity: Convex.AffineVexity()
current status: not yet solved

In [ ]:
@time solve!(prob, MosekSolver())

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] Array(::Type{Int64}, ::Int64) at ./deprecated.jl:57
 [3] conic_form!(::Convex.SDPConstraint, ::Convex.UniqueConicForms) at /Users/Guille/.julia/v0.6/Convex/src/constraints/sdp_constraints.jl:52
 [4] conic_form!(::Convex.Problem, ::Convex.UniqueConicForms) at /Users/Guille/.julia/v0.6/Convex/src/problems.jl:101
 [5] conic_problem(::Convex.Problem) at /Users/Guille/.julia/v0.6/Convex/src/problems.jl:124
 [6] #solve!#25(::Bool, ::Bool, ::Bool, ::Function, ::Convex.Problem) at /Users/Guille/.julia/v0.6/Convex/src/solution.jl:25
 [7] #solve!#24(::Array{Any,1}, ::Function, ::Convex.Problem, ::Mosek.MosekSolver) at /Users/Guille/.julia/v0.6/Convex/src/solution.jl:13
 [8] solve!(::Convex.Problem, ::Mosek.MosekSolver) at /Users/Guille/.julia/v0.6/Convex/src/solution.jl:12
 [9] include_string(::String, ::String) at ./loading.jl:522
 [10] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/Guille/.julia/v0.6/IJulia/src/exec

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 811803          
  Cones                  : 0               
  Scalar variables       : 12              
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
